In [1]:
from numpy import array, linspace
from sklearn.neighbors.kde import KernelDensity
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import argrelextrema


class OneDimensionalGaussianKernel():
    def __init__(self, bandwidth=1):
        self.bandwidth = bandwidth
        self.n_components = 0

    def fit(self, X):
        self.kde = KernelDensity(kernel='gaussian', bandwidth=self.bandwidth).fit(X)
        self.line_min = X.min()
        self.line_max = X.max()
        s = linspace(self.line_min, self.line_max)
        e = self.kde.score_samples(s.reshape(-1, 1))
        self.mins, self.maxs = argrelextrema(e, np.less)[0], argrelextrema(e, np.greater)[0]
        self.mins, self.maxs = s[self.mins], s[self.maxs]

        # self.show()

    def predict(self, X):
        mins = []
        mins.append(float('-inf'))
        [mins.append(m) for m in self.mins]
        mins.append(float('inf'))

        predicted = []
        for x in X:
            index = -1
            pre_min = None
            for min in mins:
                if pre_min is not None:
                    if pre_min < x and x <= min:
                        break

                pre_min = min
                index += 1

            predicted.append(index)

        self.n_components = max(predicted)+1

        return predicted

    def show(self):
        s = linspace(self.line_min, self.line_max)
        e = self.kde.score_samples(s.reshape(-1, 1))

        # draw data
        plt.figure(figsize=(5, 5))
        plt.scatter(s, e, marker='o', s=25, edgecolor='k')
        plt.show()

        # find min and max
        mi, ma = argrelextrema(e, np.less)[0], argrelextrema(e, np.greater)[0]
        print("Minima:", s[mi])
        print("Maxima:", s[ma])

        plt.plot(s[:mi[0] + 1], e[:mi[0] + 1], 'r',
                 s[mi[0]:mi[1] + 1], e[mi[0]:mi[1] + 1], 'g',
                 s[mi[1]:], e[mi[1]:], 'b',
                 s[ma], e[ma], 'go',
                 s[mi], e[mi], 'ro')

        plt.show()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.neighbors.kde module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
from numpy import array, linspace
from sklearn.neighbors.kde import KernelDensity
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import argrelextrema

class MultiDimensionalGaussianKernel():
    def __init__(self, bandwidth=1):
        self.bandwidth = bandwidth
        self.dict_variables = {}

    def fit(self, X):
        column_len = X.shape[1]
        for i in range(column_len):
            x = X[:, [i]]
            gk = OneDimensionalGaussianKernel(self.bandwidth)
            gk.fit(x)

            self.dict_variables[i] = gk

        for i, gk in self.dict_variables.items():
            print(i, gk)

    def predict(self, X):
        column_len = X.shape[1]
        predicted_list = []
        for i in range(column_len):
            x = X[:, [i]]
            gk = self.dict_variables[i]
            predicted = gk.predict(x)
            predicted_list.append(predicted)

        predicted = list(zip(*predicted_list))
        predicted = [str(x) for x in predicted]

        return predicted

In [3]:
import time
import warnings

import pandas as pd
import numpy as np
from sklearn import cluster, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice


class Clustering_Alg:
    def __init__(self):
        self.datasets = []
        self.selected_clustering_algorithms = []
        self.clustering_algorithms = {}
        self.clustering_variables = []
        self.scaler = StandardScaler()

    def data_scaler(self, data):
        temp_scaler = StandardScaler()
        temp_scaler.mean_ = self.scaler.mean_
        temp_scaler.var_ = self.scaler.var_
        temp_scaler.n_samples_seen_ = self.scaler.n_samples_seen_
        temp_scaler.scale_ = self.scaler.scale_
        return temp_scaler.transform(data)

    def set_algs(self, algs):
        """
        :param algs: e.g., )'MiniBatchKMeans'
                            'AffinityPropagation'
                            'MeanShift'
                            'SpectralClustering'
                            'Ward'
                            'AgglomerativeClustering'
                            'DBSCAN'
                            'OPTICS'
                            'Birch'
                            'GaussianMixture'
                            'OneDGaussianKernel'
        """
        self.selected_clustering_algorithms.append(algs)

    def set_data(self, X_data, y_data, clustering_variables):
        # data that was assigned as clustering_variables are used for clustering
        self.clustering_variables = clustering_variables

        self.X_data_all = X_data

        X_clustring_data = X_data[clustering_variables]
        y_clustring_data = y_data

        data_cl = (  # First data set
                        (  # X predictors [X1, X2]
                            # e.g., ) np.array([[10, 20], [20, 30], [11, 11], [11, 24], [25, 36], [12, 11], [30, 20]]),
                            X_clustring_data,
                            # Y target [Y]
                            # e.g., ) np.array([1, 0, 1, 1, 1, 0, 1])
                            y_clustring_data
                        ),
                        {  # Algorithm Parameters
                            # 'damping': 0.77, 'preference': -240,
                            # 'quantile': 0.2, 'n_clusters': 2, 'min_samples': 20, 'xi': 0.25
                        }
                    )

        self.datasets.append(data_cl)

    def set_base(self, parameter):
        n_clusters = 10
        bandwidth = 0.05

        if self.selected_clustering_algorithms[0] == 'OneDGaussianKernel' or \
           self.selected_clustering_algorithms[0] == 'MultiDGaussianKernel':
            bandwidth = parameter
        else:
            n_clusters = parameter

        self.default_base = {'quantile': .3,
                             'eps': .3,
                             'damping': .9,
                             'preference': -200,
                             'n_neighbors': 10,
                             'n_clusters': n_clusters,
                             'min_samples': 10,
                             'xi': 0.05,
                             'min_cluster_size': 0.1,
                             'bandwidth':bandwidth}

    def get_selected_clustering_alg(self):
        return self.clustering_algorithms[self.selected_clustering_algorithms[0]]

    def get_clustered_data(self, alg):
        """
        This returns clustered data according to a selected algorithm
        :param alg: a selected algorithm
        :return: clustered data
        """

        algorithm = self.clustering_algorithms[alg]
        # print(self.datasets)
        re_X_data = {}
        for i_dataset, (dataset, algo_params) in enumerate(self.datasets):
            X = dataset
            if isinstance(X, pd.DataFrame):
                for i in range(len(X.index)):
                    data_id = algorithm.labels_[i]
                    if data_id not in re_X_data:
                        re_X_data[data_id] = X.iloc[i].transpose()
                    else:
                        re_X_data[data_id] = pd.concat((re_X_data[data_id], X.iloc[i]), axis=1)

                for key in re_X_data:
                    re_X_data[key] = re_X_data[key].transpose()

            elif not isinstance(X, pd.DataFrame):
                for i in range(len(X)):
                    data_id = algorithm.labels_[i]
                    if data_id not in re_X_data:
                        re_X_data[data_id] = np.array([X[i]])
                    else:
                        re_X_data[data_id] = np.concatenate((re_X_data[data_id], [X[i]]), axis=0)

            return re_X_data

    def get_clustered_data_XY(self, alg):
        """
        This returns clustered data according to a selected algorithm
        :param alg: a selected algorithm
        :return: clustered data
        """

        algorithm = self.clustering_algorithms[alg]
        # print(self.datasets)
        re_X_data = {}
        re_y_data = {}
        re_data = {}
        for i_dataset, (dataset, algo_params) in enumerate(self.datasets):
            X, y = dataset
            if isinstance(X, pd.DataFrame):
                for i in range(len(X.index)):
                    data_id = algorithm.labels_[i]
                    if data_id not in re_X_data:
                        re_X_data[data_id] = self.X_data_all.iloc[i].transpose()
                        re_y_data[data_id] = y.iloc[i]
                        re_data[data_id] = pd.concat([self.X_data_all.iloc[i], y.iloc[i]], axis=0)
                    else:
                        re_X_data[data_id] = pd.concat((re_X_data[data_id], self.X_data_all.iloc[i]), axis=1)
                        re_y_data[data_id] = pd.concat((re_y_data[data_id], y.iloc[i]), axis=1)
                        re_data[data_id] = pd.concat((re_X_data[data_id], re_y_data[data_id]), axis=0)

                for key in re_X_data:
                    re_X_data[key] = re_X_data[key].transpose()
                    re_y_data[key] = re_y_data[key].transpose()
                    re_data[key] = re_data[key].transpose()

            elif not isinstance(X, pd.DataFrame):
                for i in range(len(X)):
                    data_id = algorithm.labels_[i]
                    if data_id not in re_X_data:
                        re_X_data[data_id] = np.array([self.X_data_all[i]])
                        re_y_data[data_id] = np.array([y[i]])
                        re_data[data_id] = np.concatenate((re_X_data[data_id], re_y_data[data_id]), axis=1)
                    else:
                        re_X_data[data_id] = np.concatenate((re_X_data[data_id], [self.X_data_all[i]]), axis=0)
                        re_y_data[data_id] = np.concatenate((re_y_data[data_id], [y[i]]), axis=0)
                        re_data[data_id] = np.concatenate((re_X_data[data_id], re_y_data[data_id]), axis=1)

            return re_data, re_X_data, re_y_data

    def run(self):

        for i_dataset, (dataset, algo_params) in enumerate(self.datasets):
            # update parameters with dataset-specific values
            params = self.default_base.copy()
            params.update(algo_params)

            X, y = dataset

            # normalize dataset for easier parameter selection
            X = self.scaler.fit_transform(X)

            print(f'mean{self.scaler.mean_}, var{self.scaler.var_}, n_samples[{self.scaler.n_samples_seen_}], scale[{self.scaler.scale_}]')

            # estimate bandwidth for mean shift
            bandwidth = cluster.estimate_bandwidth(X, quantile=params['quantile'])

            # connectivity matrix for structured Ward
            connectivity = kneighbors_graph(X, n_neighbors=params['n_neighbors'], include_self=False)

            # make connectivity symmetric
            connectivity = 0.5 * (connectivity + connectivity.T)

            # ============
            # Create cluster objects
            # ============
            for alg in self.selected_clustering_algorithms:
                if alg is 'MiniBatchKMeans':
                    self.clustering_algorithms['MiniBatchKMeans'] = cluster.MiniBatchKMeans(n_clusters=params['n_clusters'])
                elif alg is 'AffinityPropagation':
                    self.clustering_algorithms['AffinityPropagation'] = cluster.AffinityPropagation(damping=params['damping'],preference=params['preference'])
                elif alg is 'MeanShift':
                    self.clustering_algorithms['MeanShift'] = cluster.MeanShift(bandwidth=bandwidth, bin_seeding=True)
                elif alg is 'SpectralClustering':
                    self.clustering_algorithms['SpectralClustering'] = cluster.SpectralClustering(n_clusters=params['n_clusters'], eigen_solver='arpack', affinity="nearest_neighbors")
                elif alg is 'Ward':
                    self.clustering_algorithms['Ward'] = cluster.AgglomerativeClustering(n_clusters=params['n_clusters'], linkage='ward', connectivity=connectivity)
                elif alg is 'AgglomerativeClustering':
                    self.clustering_algorithms['AgglomerativeClustering'] = cluster.AgglomerativeClustering(linkage="average", affinity="cityblock", n_clusters=params['n_clusters'], connectivity=connectivity)
                elif alg is 'DBSCAN':
                    self.clustering_algorithms['DBSCAN'] = cluster.DBSCAN(eps=params['eps'])
                elif alg is 'OPTICS':
                    self.clustering_algorithms['OPTICS'] = cluster.OPTICS(min_samples=params['min_samples'], xi=params['xi'], min_cluster_size=params['min_cluster_size'])
                elif alg is 'Birch':
                    self.clustering_algorithms['Birch'] = cluster.Birch(n_clusters=params['n_clusters'])
                elif alg is 'GaussianMixture':
                    self.clustering_algorithms['GaussianMixture'] = mixture.GaussianMixture(n_components=params['n_clusters'], covariance_type='full')
                elif alg is 'OneDGaussianKernel':
                    self.clustering_algorithms['OneDGaussianKernel'] = OneDimensionalGaussianKernel(bandwidth=params['bandwidth'])
                elif alg is 'MultiDGaussianKernel':
                    self.clustering_algorithms['MultiDGaussianKernel'] = MultiDimensionalGaussianKernel(bandwidth=params['bandwidth'])


            # self.clustering_algorithms['MiniBatchKMeans'] = cluster.MiniBatchKMeans(n_clusters=params['n_clusters'])
            # self.clustering_algorithms['AffinityPropagation'] = cluster.AffinityPropagation(damping=params['damping'], preference=params['preference'])
            # self.clustering_algorithms['MeanShift'] = cluster.MeanShift(bandwidth=bandwidth, bin_seeding=True)
            # self.clustering_algorithms['SpectralClustering'] = cluster.SpectralClustering(n_clusters=params['n_clusters'], eigen_solver='arpack', affinity="nearest_neighbors")
            # self.clustering_algorithms['Ward'] = cluster.AgglomerativeClustering(n_clusters=params['n_clusters'], linkage='ward', connectivity=connectivity)
            # self.clustering_algorithms['AgglomerativeClustering'] = cluster.AgglomerativeClustering(linkage="average", affinity="cityblock", n_clusters=params['n_clusters'], connectivity=connectivity)
            # self.clustering_algorithms['DBSCAN'] = cluster.DBSCAN(eps=params['eps'])
            # self.clustering_algorithms['OPTICS'] = cluster.OPTICS(min_samples=params['min_samples'], xi=params['xi'], min_cluster_size=params['min_cluster_size'])
            # self.clustering_algorithms['Birch'] = cluster.Birch(n_clusters=params['n_clusters'])
            # self.clustering_algorithms['GaussianMixture'] = mixture.GaussianMixture(n_components=params['n_clusters'], covariance_type='full')

            for name, algorithm in self.clustering_algorithms.items():
                t0 = time.time()

                # catch warnings related to kneighbors_graph
                with warnings.catch_warnings():
                    warnings.filterwarnings(
                        "ignore",
                        message="the number of connected components of the " +
                        "connectivity matrix is [0-9]{1,2}" +
                        " > 1. Completing it to avoid stopping the tree early.",
                        category=UserWarning)
                    warnings.filterwarnings(
                        "ignore",
                        message="Graph is not fully connected, spectral embedding" +
                        " may not work as expected.",
                        category=UserWarning)
                    algorithm.fit(X)

                t1 = time.time()
                if hasattr(algorithm, 'labels_'):
                    y_pred = algorithm.labels_.astype(np.int)
                else:
                    y_pred = algorithm.predict(X)
                    algorithm.labels_ = y_pred

In [4]:
import time
import warnings

import numpy as np
import matplotlib.pyplot as plt
from sklearn import cluster, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice


class cluster_test:
    def __init__(self):
        self.datasets = []

    def set_data(self, X_data, y_data):
        new_data = (  # First data set
                        (  # X predictors [X1, X2]
                            # e.g., ) np.array([[10, 20], [20, 30], [11, 11], [11, 24], [25, 36], [12, 11], [30, 20]]),
                            X_data,
                            # Y target [Y]
                            # e.g., ) np.array([1, 0, 1, 1, 1, 0, 1])
                            y_data
                        ),
                        {  # Algorithm Parameters
                            # 'damping': 0.77, 'preference': -240,
                            # 'quantile': 0.2, 'n_clusters': 2, 'min_samples': 20, 'xi': 0.25
                        }
                    )

        self.datasets.append(new_data)

        self.default_base = {'quantile': .3,
                             'eps': .3,
                             'damping': .9,
                             'preference': -200,
                             'n_neighbors': 10,
                             'n_clusters': 5,
                             'min_samples': 10,
                             'xi': 0.05,
                             'min_cluster_size': 0.1}

    def run(self):
        plt.figure(figsize=(9 * 2 + 3, len(self.datasets)*2))
        # plt.subplots_adjust(left=.02, right=.98, bottom=.001, top=.96, wspace=.05, hspace=.01)

        plot_num = 1

        for i_dataset, (dataset, algo_params) in enumerate(self.datasets):
            # update parameters with dataset-specific values
            params = self.default_base.copy()
            params.update(algo_params)

            X, y = dataset

            # normalize dataset for easier parameter selection
            X = StandardScaler().fit_transform(X)

            # estimate bandwidth for mean shift
            bandwidth = cluster.estimate_bandwidth(X, quantile=params['quantile'])

            # connectivity matrix for structured Ward
            connectivity = kneighbors_graph(X, n_neighbors=params['n_neighbors'], include_self=False)

            # make connectivity symmetric
            connectivity = 0.5 * (connectivity + connectivity.T)

            # ============
            # Create cluster objects
            # ============
            ms = cluster.MeanShift(bandwidth=bandwidth, bin_seeding=True)
            two_means = cluster.MiniBatchKMeans(n_clusters=params['n_clusters'])
            ward = cluster.AgglomerativeClustering(n_clusters=params['n_clusters'], linkage='ward', connectivity=connectivity)
            spectral = cluster.SpectralClustering(n_clusters=params['n_clusters'], eigen_solver='arpack', affinity="nearest_neighbors")
            dbscan = cluster.DBSCAN(eps=params['eps'])
            optics = cluster.OPTICS(min_samples=params['min_samples'], xi=params['xi'], min_cluster_size=params['min_cluster_size'])
            affinity_propagation = cluster.AffinityPropagation(damping=params['damping'], preference=params['preference'])
            average_linkage = cluster.AgglomerativeClustering(linkage="average", affinity="cityblock", n_clusters=params['n_clusters'], connectivity=connectivity)
            birch = cluster.Birch(n_clusters=params['n_clusters'])
            gmm = mixture.GaussianMixture(n_components=params['n_clusters'], covariance_type='full')

            clustering_algorithms = (
                ('MiniBatchKMeans', two_means),
                ('AffinityPropagation', affinity_propagation),
                ('MeanShift', ms),
                ('SpectralClustering', spectral),
                ('Ward', ward),
                ('AgglomerativeClustering', average_linkage),
                ('DBSCAN', dbscan),
                ('OPTICS', optics),
                ('Birch', birch),
                ('GaussianMixture', gmm)
            )

            for name, algorithm in clustering_algorithms:
                t0 = time.time()

                # catch warnings related to kneighbors_graph
                with warnings.catch_warnings():
                    warnings.filterwarnings(
                        "ignore",
                        message="the number of connected components of the " +
                        "connectivity matrix is [0-9]{1,2}" +
                        " > 1. Completing it to avoid stopping the tree early.",
                        category=UserWarning)
                    warnings.filterwarnings(
                        "ignore",
                        message="Graph is not fully connected, spectral embedding" +
                        " may not work as expected.",
                        category=UserWarning)
                    algorithm.fit(X)

                t1 = time.time()
                if hasattr(algorithm, 'labels_'):
                    y_pred = algorithm.labels_.astype(np.int)
                else:
                    y_pred = algorithm.predict(X)

                plt.subplot(len(self.datasets), len(clustering_algorithms), plot_num)
                if i_dataset == 0:
                    plt.title(name, size=9)

                colors = np.array(list(islice(cycle(['#377eb8', '#ff7f00', '#4daf4a',
                                                     '#f781bf', '#a65628', '#984ea3',
                                                     '#999999', '#e41a1c', '#dede00']),
                                              int(max(y_pred) + 1))))
                # add black color for outliers (if any)
                colors = np.append(colors, ["#000000"])
                plt.scatter(X[:, 0], X[:, 1], s=10, color=colors[y_pred])

                plt.xlim(-2.5, 2.5)
                plt.ylim(-2.5, 2.5)
                plt.xticks(())
                plt.yticks(())
                plt.text(.99, .01, ('%.2fs' % (t1 - t0)).lstrip('0'),
                         transform=plt.gca().transAxes, size=8,
                         horizontalalignment='right')
                plot_num += 1

        plt.show()

In [5]:
import subprocess
import os
import json
import sys
import pandas as pd
import concurrent.futures
import time

class DMP_runner():
    def __init__(self, output):
        self.output = output
        self.my_df = pd.DataFrame()
        self.__model = ''
        self.__evidence = ''

    def setModel(self, sbn):
        with open(sbn, 'r') as file:
            self.__model += file.read()

    def addEvidence(self, nodeName, value):
        s = '\ndefineEvidence({}, {});'.format(nodeName, value)
        self.__evidence += s

    def runDMP(self):
        model = self.__model + self.__evidence + '\nrun(DMP);'
        self.run(model)
        self.__evidence = ''

    def saveResult(self):
        ###############################
        # Save reasoned results
        nodes = []
        with open(self.output) as json_file:
            net = json.load(json_file)
            for node in net:
                for obj, contents in node.items():
                    nodes.append(obj)

        df = pd.DataFrame(columns=nodes)

        row_data = {}
        with open(self.output) as json_file:
            net = json.load(json_file)
            for node in net:
                for obj, contents in node.items():
                    if 'marginal' in contents:
                        row_data[obj] = contents['marginal']['MU']
                    elif 'evidence' in contents:
                        row_data[obj] = contents['evidence']['mean']

        df = df.append(row_data, ignore_index=True)
        self.my_df = self.my_df.append(df, ignore_index=True)
        return self.my_df

    def run(self, model):
        # create a hybrid model script to be executed in dmp.jar
        cwd = os.path.dirname(os.path.realpath(__file__))
        dmp_args = " -b "
        dmp_args += "\"" + model + "\""
        dmp_args += " -p "
        dmp_args += "\"" + self.output + "\""

        # supervisor call to DMP (command line)
        proc = subprocess.Popen("java -jar " + cwd + "/dmp.jar" + dmp_args)

        proc.wait()

        (stdout, stderr) = proc.communicate()

        if proc.returncode != 0:
            print(stderr)
            sys.exit()
        else:
            # print("success")
            pass

    ##############################################################
    # sampling
    def sampling(self, sbn, size):
        time.sleep(0.1)
        model = ""

        with open(sbn, 'r') as file:
            model += file.read()
        model += " run(LW, arg(1,1));"

        for i in range(size):
            print("   Sampling {:f} % ********************************".format(100*i/size))

            # Run a BN once using LW
            self.run(model)

        return self.saveResult()

    def sampling_by_thread(self, sbn, size, size_thread=50):
        # size_thread = 4
        distributed_size = size
        ts = time.time()

        with concurrent.futures.ThreadPoolExecutor(max_workers=size_thread) as executor:
            executor.map(self.sampling(sbn, distributed_size), range(size_thread))

        # with concurrent.futures.ProcessPoolExecutor(max_workers=size_thread) as executor:
        #     executor.map(self.sampling(sbn, distributed_size), range(size_thread))

        print("== sampling_by_thread end: Time {} ==============================".format(time.time() - ts))

        return self.my_df

    ##############################################################
    # read output from DMP
    def read_results(self, data_file_name):
        with open(data_file_name) as json_file:
            net = json.load(json_file)
            for node in net:
                for obj, contents in node.items():
                    print('node: ' + obj)
                    for attr, values in contents.items():
                        print(" " + attr + ': ' + str(values))


In [6]:
import subprocess
import os
import json
import sys


class HML_runner():
    def __init__(self, model_structure=None):
        self.model_structure = model_structure
        self.csv = ""
        self.output = ""

    def run(self, csv, output):
        self.csv = csv
        self.output = output

        # create a hybrid model script to be executed in dmp.jar
        cwd = os.path.dirname(os.path.realpath(__file__))
        dmp_args = " -b "
        dmp_args += "\"" + self.model_structure + "\""
        dmp_args += " -c "
        dmp_args += "\"" + self.csv + "\""
        dmp_args += " -p "
        dmp_args += "\"" + self.output + "\""

        # supervisor call to HML (command line)
        # print("current working directory: " + cwd)
        proc = subprocess.Popen("java -jar " + cwd + "/HML.jar" + dmp_args)

        proc.wait()

        (stdout, stderr) = proc.communicate()

        if proc.returncode != 0:
            print(stderr)
            sys.exit()
        else:
            # print("success")
            with open(self.output, 'r') as file:
                self.model_learned = file.read()
            return self.model_learned

    def make_Model(self, child, parents):
        self.model_structure = ""
        for nodeName in parents:
            self.model_structure += self.creat_BN_parent_node(nodeName)
        self.model_structure += self.creat_BN_child_Node(child, parents)
        # print(self.model_structure)
        return self.model_structure

    def creat_BN_parent_node(self, nodeName):
        s = "defineNode({}, des);".format(nodeName)
        s += "{ defineState(Continuous);"
        s += "  p( {} ) = NormalDist(  0, 0.000001 );".format(nodeName)
        s += "}"
        return s

    def creat_BN_child_Node(self, nodeName, parents):
        p = ", ".join(parents)
        e = "+ ".join(parents)
        s = "defineNode({}, des);".format(nodeName)
        s += "{ defineState(Continuous);"
        s += "  p( {} | {} ) = {} + NormalDist(  0, 0.000001 );".format(nodeName, p, e)
        s += "}"
        return s

    # read output from DMP
    def read_results(self, data_file_name):
        with open(data_file_name) as json_file:
            net = json.load(json_file)
            for node in net:
                for obj, contents in node.items():
                    print('node: ' + obj)
                    for attr, values in contents.items():
                        print(" " + attr + ': ' + str(values))


In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
import pandas as pd
import numpy as np
import time
import json


class RegressionML():
    def __init__(self, metrics='R2'):
        # Set metrics
        if metrics == 'R2':
            self.score = r2_score
        elif metrics == 'MAE':
            self.score = mean_absolute_error

    def show_results(self, y_predicted, y_actual=None, ML_Alg=None, cv=5):
        if y_actual is None or len(y_actual) == 0:
            return 0

        # Score metric: R^2
        """ The coefficient R^2 is defined as (1 - u/v), where u is the residual
            sum of squares ((y_true - y_pred) ** 2).sum() and v is the total
            sum of squares ((y_true - y_true.mean()) ** 2).sum().
            The best possible score is 1.0 and it can be negative (because the
            model can be arbitrarily worse). A constant model that always
            predicts the expected value of y, disregarding the input features,
            would get a R^2 score of 0.0."""

        # Training Cross Validation Accuracy Score
        # if ML_Alg != None:
        #     val_scores = cross_val_score(ML_Alg, X_train, np.ravel(y_train), cv=cv)
        #     print(f'Training Cross Validation Score({val_scores.mean()}):', val_scores)

        # Testing Accuracy Score
        r2 = self.score(y_actual, y_predicted)
        # if len(y_actual) > 0 and len(y_predicted) > 0:
        #     print("Testing R^2 Accuracy Score:", r2)

        return r2

    def DecisionTreeClassifier_run(self, X_train, y_train, X_test, y_actual=None):
        ml_model = self.DecisionTreeClassifier_ML(X_train, y_train)
        return self.prediction(ml_model, X_test, y_actual)

    def DecisionTreeClassifier_ML(self, X_train, y_train):
        # print("*** Decision Tree Regression ***")
        ML_Alg = DecisionTreeClassifier(max_depth=6)
        ml_model = ML_Alg.fit(X_train, y_train.ravel())
        return ml_model

    def GaussianNB_run(self, X_train, y_train, X_test, y_actual=None):
        ml_model = self.GaussianNB_ML(X_train, y_train)
        return self.prediction(ml_model, X_test, y_actual)

    def GaussianNB_ML(self, X_train, y_train):
        # print("*** Gaussian NB ***")
        ML_Alg = GaussianNB()
        ml_model = ML_Alg.fit(X_train, np.ravel(y_train))
        return ml_model

    def RandomForestRegressor_run(self, X_train, y_train, X_test, y_actual=None):
        ml_model = self.RandomForestRegressor_ML(X_train, y_train)
        return self.prediction(ml_model, X_test, np.ravel(y_actual))

    def RandomForestRegressor_ML(self, X_train, y_train):
        # print("*** Random Forest Regression ***")
        ML_Alg = RandomForestRegressor(n_estimators=100)
        ml_model = ML_Alg.fit(X_train, np.ravel(y_train))
        return ml_model


    def LinearRegressor_run(self, X_train, y_train, X_test, y_actual=None):
        ml_model = self.LinearRegressor_ML(X_train, y_train)
        return self.prediction(ml_model, X_test, y_actual)

    def LinearRegressor_ML(self, X_train, y_train):
        # print("*** Random LinearRegression Regression ***")
        ML_Alg = LinearRegression()
        ml_model = ML_Alg.fit(X_train, np.ravel(y_train))
        return ml_model


    def GradientBoostingRegressor_run(self, X_train, y_train, X_test, y_actual=None):
        ml_model = self.GradientBoostingRegressor_ML(X_train, y_train)
        return self.prediction(ml_model, X_test, y_actual)

    def GradientBoostingRegressor_ML(self, X_train, y_train):
        # print("*** Gradient Boosting Regression ***")
        ML_Alg = GradientBoostingRegressor(n_estimators=1000,
                                           learning_rate=0.1,
                                           subsample=0.5,
                                           max_depth=1,
                                           random_state=0)
        ml_model = ML_Alg.fit(X_train, np.ravel(y_train))
        return ml_model

    def GaussianProcessRegressor_run(self, X_train, y_train, X_test, y_actual=None):
        ml_model = self.GaussianProcessRegressor_ML(X_train, y_train)
        return self.prediction(ml_model, X_test, y_actual)

    def GaussianProcessRegressor_ML(self, X_train, y_train):
        # print("*** Gaussian Process Regression ***")
        kernel = DotProduct() + WhiteKernel()
        ML_Alg = GaussianProcessRegressor(kernel=kernel, random_state=0)
        ml_model = ML_Alg.fit(X_train, np.ravel(y_train))
        return ml_model

    def prediction(self, ml_model, X_test, y_actual=None):
        y_predicted = ml_model.predict(X_test)
        return y_predicted, self.show_results(y_predicted, y_actual)

    def run_with_evidence_and_check_prediction(self, dmp, model, data, y_actual):
        predicted = []
        i = 0
        for index, rows in data.iterrows():
            keys = rows.keys()
            sbn = model
            y_name = y_actual.columns[0]
            y_value = y_actual.iloc[i, 0]
            for k in keys:
                sbn += "defineEvidence({}, {});".format(k, rows.get(k))
            sbn += "run(DMP);"
            dmp.run(sbn)

            # check prediction
            # print("================ Prediction results from BN ================")
            # print(dmp.output)
            with open(dmp.output) as json_file:
                net = json.load(json_file)
                for node in net:
                    for obj, contents in node.items():
                        # print('node: ' + obj)
                        mean = None
                        for attr, values in contents.items():
                            # print(" " + attr + ': ' + str(values))
                            if attr == "marginal":
                                mean = values["MU"]

                        if obj == y_name:
                            # print("================================")
                            # print("{} : Predicted {} : Actual {}".format(y_name, mean, y_value))
                            predicted.append(float(mean))

            i += 1

        return predicted

    def ContinuousBNRegressor_run(self, name, X_train, y_train, X_test, y_actual=None, show=True):
        ssbn = self.ContinuousBNRegressor_ML(name, X_train, y_train)
        return self.ContinuousBNRegressor_prediction(name, ssbn, X_test, y_actual, show)

    def ContinuousBNRegressor_ML(self, name, X_train, y_train):
        # print("*** Continuous BN Regressor  ***")
        #############################
        # Make a csv data file
        csv = r'../TestData/{}_for_test.csv'.format(name)
        # csv = "E:/SW-Posco2019/DATAETL/TestData/big_data_1000000.csv"
        output = r'../Output_BN/{}_ssbn.txt'.format(name)

        df_col = pd.concat([X_train, y_train], axis=1)
        df_col.to_csv(csv, index=None)

        #############################
        # Run MEBN learning
        ts = time.time()
        hml = HML_runner()

        # Make a V-BN model
        parents = []
        child = y_train.columns[0]
        for nodeName in X_train.columns:
            parents.append(nodeName)

        hml.make_Model(child, parents)

        # Run MEBN learning
        ssbn = hml.run(csv, output)

        # print("== Mahcine learning end: Time {} ==============================".format(time.time()-ts))
        return ssbn

    def ContinuousBNRegressor_prediction(self, name, ssbn, X_test, y_actual=None, show=True):
        #############################
        # Prediction
        ts = time.time()
        output = r"../Output_BN/{}_bn_output.json".format(name)
        dmp = DMP_runner(output)

        y_predicted = self.run_with_evidence_and_check_prediction(dmp, ssbn, X_test, y_actual)

        # print("== BN was completed : Time {} ==============================".format(time.time()-ts))

        if show == True and len(y_actual) > 1:
            return y_predicted, self.show_results(y_predicted, y_actual)
        else:
            return y_predicted, self.show_results(y_predicted, y_actual=None)

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
import threading
from statistics import mean
import math
import warnings
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from datetime import datetime

class MLModelFamily:
    """
    This class is used as a structure of machine learning family
    """

    def __init__(self, id=0):
        self.id = id
        self.models = {}
        self.models = {}
        self.data = {}

    def add_CL(self, cl_alg):
        self.models[cl_alg] = {}
        self.data[cl_alg] = {}

    def get_CL(self):
        return self.models

    def del_CL(self, cl_alg):
        del self.models[cl_alg]
        del self.data[cl_alg]

    def add_CL_param(self, cl_alg, parameters):
        self.models[cl_alg][parameters] = {}
        self.data[cl_alg][parameters] = {}

    def del_CL_param(self, cl_alg, parameters):
        del self.models[cl_alg][parameters]
        del self.data[cl_alg][parameters]

    def add_CL_model(self, cl_alg, parameters, cl):
        if 'CL_MODEL' not in self.models[cl_alg][parameters]:
            self.models[cl_alg][parameters]['CL_MODEL'] = {}

        self.models[cl_alg][parameters]['CL_MODEL'] = cl

    def set_CL_model_avg_r2(self, cl_alg, parameters, avg_r2):
        self.models[cl_alg][parameters]['AVG_R2'] = avg_r2

    def get_CL_model_avg_r2(self, cl_alg, parameters):
        return self.models[cl_alg][parameters]['AVG_R2']

    def set_CL_total_avg_r2(self, cl_alg, avg_r2):
        self.models[cl_alg]['TOTAL_AVG_R2'] = avg_r2

    def get_CL_total_avg_r2(self, cl_alg):
        return self.models[cl_alg]['TOTAL_AVG_R2']

    def set_CL_data(self, cl_alg, parameters, cur_cluster, data_name, data):
        if cur_cluster not in self.data[cl_alg][parameters]:
            self.data[cl_alg][parameters][cur_cluster] = {}
        self.data[cl_alg][parameters][cur_cluster][data_name] = data

    def add_SL(self, cl_alg, parameters, cur_cluster, prediction_alg):
        if cur_cluster not in self.models[cl_alg][parameters]:
            self.models[cl_alg][parameters][cur_cluster] = {}

        self.models[cl_alg][parameters][cur_cluster][prediction_alg] = {}

    def del_SL(self, cl_alg, parameters, cur_cluster, prediction_alg):
        del self.models[cl_alg][parameters][cur_cluster][prediction_alg]

    def add_SL_model(self, cl_alg, parameters, cur_cluster, prediction_alg, model):
        self.models[cl_alg][parameters][cur_cluster][prediction_alg]['SL_MODEL'] = model

    def set_SL_model_r2(self, cl_alg, parameters, cur_cluster, prediction_alg, r2):
        if math.isnan(r2):
            # In some cases, the training data has the size of 1, then R2 becomes NaN.
            warnings.warn(f'{cl_alg}.{parameters}.{cur_cluster}.{prediction_alg}: The prediction result was NaN.')
            self.models[cl_alg][parameters][cur_cluster][prediction_alg]['R2'] = -10
        else:
            self.models[cl_alg][parameters][cur_cluster][prediction_alg]['R2'] = r2

    def get_SL_model_r2(self, cl_alg, parameters, cur_cluster, prediction_alg):
        return self.models[cl_alg][parameters][cur_cluster][prediction_alg]['R2']

    def get_sl_models(self):
        """
        This returns all selected supervised learning models in the ML model family
        :return: a dictionary for pairs of supervised learning models
        e.g., ) {0: [SL model object], 1: [SL model object], ...}
        0 and 1 stand for the cluster id
        """

        cl = next(iter(self.models.values()))
        cl_model = next(iter(cl.values()))
        result = {}

        for cluster_id, value in cl_model.items():
            if cluster_id is not 'CL_MODEL' and cluster_id is not 'AVG_R2':
                sl_model = next(iter(value.values()))
                result[cluster_id] = sl_model

        return result

    def get_sl_model(self, cluster_id):
        """
        This returns a selected supervised learning model associated with a cluster id
        :param cluster_id: the index of a cluster in a clustering model in the ML model family
        :return: a SL model name, a SL model object
        """

        cl = next(iter(self.models.values()))
        cl_model = next(iter(cl.values()))

        for key, value in cl_model.items():
            if key is not 'CL_MODEL' and key is not 'AVG_R2':
                if key == cluster_id:
                    sl_model = next(iter(value.values()))
                    return list(value.keys())[0], sl_model['SL_MODEL']

        return None, None

    def get_cl_model(self):
        """
        This returns the clustering algorithm class
        """

        cl = next(iter(self.models.values()))
        cl_model = next(iter(cl.values()))
        if isinstance(cl_model['CL_MODEL'], Clustering_Alg):
            return cl_model['CL_MODEL']
        else:
            return None

    def get_clustering_alg(self):
        """
        This returns a selected clustering algorithm
        """

        cl_model = self.get_cl_model()
        if cl_model is not None:
            return cl_model.get_selected_clustering_alg()
        else:
            return None

    def get_CL_models(self, cl_alg, parameters):
        cl_models = {}
        for key, value in self.models[cl_alg][parameters].items():
            if key is not 'CL_MODEL' and key is not 'AVG_R2':
                cl_models[key] = value

        return cl_models

    def set_high_CL_model(self, cl_alg, alg_high):
        self.models[cl_alg]['HIGH_CL'] = alg_high

    def get_high_CL_model(self, cl_alg):
        return self.models[cl_alg]['HIGH_CL']

    def get_clustered_data_by_id(self, cluster_id):
        """
        This returns a clustered data associated with a cluster id
        :param cluster_id: the index of a cluster in a clustering model in the ML model family
        :return: X data, y data
        """

        cl = next(iter(self.data.values()))
        cl_data= next(iter(cl.values()))

        X, y = None, None
        for key, data_dict in cl_data.items():
            if key == cluster_id:
                X = data_dict['x_train']
                y = data_dict['y_train']
                break

        return X, y

    def get_clustered_all_data_by_id(self, cluster_id):
        X, y = self.get_clustered_data_by_id(cluster_id)
        data = pd.concat((X, y), axis=1)
        return data

class DataClusterBasedMachineLearning:
    """
    Data Cluster based Machine Learning (DC-ML)
    """

    def __init__(self, data_x, data_y, clustering_variables, clustering_algs, prediction_algs, metrics='MAE'):
        """
        :param data: A training data set D
        :param clustering_alg: A set of clustering algorithms C
        :param prediction_alg: A set of prediction algorithms P
        :param max_clusters: A maximum number of clusters m
        """
        self.data_x = data_x
        self.data_y = data_y
        self.clustering_variables = clustering_variables
        self.clustering_algs = clustering_algs
        self.prediction_algs = prediction_algs

        # Initialize an ML Model Family
        # It contains all ML models (i.e., Clustering Models (CL) and their Supervised Learning (SL) Models)
        self.ml_family = MLModelFamily()

        # For experiment
        self.is_experiment = True
        self.set_metrics(metrics)

    def set_metrics(self, metrics):
        # Set metrics
        self.metrics = metrics
        if metrics == 'R2':
            self.score = r2_score
        elif metrics == 'MAE':
            self.score = mean_absolute_error

    def min_or_max(self):
        if self.metrics == 'R2':
            return max, float('-inf')
        elif self.metrics == 'MAE':
            return min, float('inf')

    def do_machine_learning(self, prediction_alg, x_train, y_train, cbn_name='temp'):
        """
        This performs common SL learning
        :param prediction_alg: A name of SL learning
        :param x_train: Training data for X variables
        :param y_train: Training data for a y variable
        :param cbn_name: A special parameter for continuous BN learning
        :return: A SL learning object
        """

        if prediction_alg is 'GradientBoostingRegressor':
            model = RegressionML(self.metrics).GradientBoostingRegressor_ML(x_train, y_train)
        elif prediction_alg is 'RandomForestRegressor':
            model = RegressionML(self.metrics).RandomForestRegressor_ML(x_train, y_train)
        elif prediction_alg is 'GaussianProcessRegressor':
            model = RegressionML(self.metrics).GaussianProcessRegressor_ML(x_train, y_train)
        elif prediction_alg is 'LinearRegression':
            model = RegressionML(self.metrics).LinearRegressor_ML(x_train, y_train)
        elif prediction_alg is 'ContinuousBNRegressor':
            model = RegressionML(self.metrics).ContinuousBNRegressor_ML(cbn_name, x_train, y_train)
        return model

    def do_prediction(self, model_name, model, X, y=None, cbn_name='temp'):
        """
        This performs prediction using a given SL learning
        :param model_name: A name of SL learning
        :param model: An object of SL learning
        :param X: Data for X variables
        :param y: Data for a y variable
        :param cbn_name: A special parameter for continuous BN learning
        :return:
        """

        if model_name is 'ContinuousBNRegressor':
            yPredicted, r2 = RegressionML(self.metrics).ContinuousBNRegressor_prediction(cbn_name, model, X, y)
        else:
            yPredicted, r2 = RegressionML(self.metrics).prediction(model, X, y)
        return yPredicted, r2

    def perform_machine_learning_alg(self, cl_alg, parameters, cur_cluster, prediction_alg, x_clustered, y_clustered):
        """
        This performs an SL algorithm
        :param cl_alg: The name of a clustering algorithm
        :param parameters:
        :param cur_cluster: The current cluster index of the clustering algorithm
        :param prediction_alg: A name of SL learning
        :param x_train: Training data for X variables
        :param x_val: Validation data for X variables
        :param y_train: Training data for a y variable
        :param y_val: Validation data for a y variable
        """

        # temporary name for CBN
        cbn_name = f'{cl_alg}_{parameters}_{cur_cluster}_{datetime.now().strftime("%m_%d_%Y %H_%M_%S")}'
        n_splits = 2
        kf = KFold(n_splits=n_splits)

        r2_avg = 0
        for train_index, val_index in kf.split(x_clustered):
            x_train, x_val = x_clustered.iloc[train_index], x_clustered.iloc[val_index]
            y_train, y_val = y_clustered.iloc[train_index], y_clustered.iloc[val_index]

            # perform ML
            model = self.do_machine_learning(prediction_alg, x_train, y_train, cbn_name=cbn_name)

            # perform prediction
            yPredicted, r2 = self.do_prediction(prediction_alg, model, x_val, y_val, cbn_name=cbn_name)
            r2_avg += r2

        r2_avg = r2_avg/n_splits
        # print(cbn_name, prediction_alg, model.feature_importances_)

        self.ml_family.add_SL_model(cl_alg, parameters, cur_cluster, prediction_alg, model)
        self.ml_family.set_SL_model_r2(cl_alg, parameters, cur_cluster, prediction_alg, r2_avg)

    def perform_machine_learning(self, cl_alg, parameters, cur_cluster, x_clustered, y_clustered):
        """
        This executes a set of SL algorithms
        :param cl_alg: The name of a clustering algorithm
        :param parameters:
        :param cur_cluster: The current cluster index of the clustering algorithm
        :param x_train: Training data for X variables
        :param x_val: Validation data for X variables
        :param y_train: Training data for a y variable
        :param y_val: Validation data for a y variable
        """

        #############################################################################
        # 1. perform common machine learning
        thread = []
        for prediction_alg in self.prediction_algs:
            self.ml_family.add_SL(cl_alg, parameters, cur_cluster, prediction_alg)

            print(f'[Thread] {cl_alg}.{parameters}.{cur_cluster} -> ML alg {prediction_alg}')

            t = threading.Thread(target=self.perform_machine_learning_alg, args=([cl_alg, parameters, cur_cluster, prediction_alg, x_clustered, y_clustered]))
            t.setDaemon(True)
            thread.append(t)

        for t in thread:
            t.start()

        for t in thread:
            t.join()

        #############################################################################
        # 2.Select a best prediction alg and remove all low-scored algorithms
        alg_high = None
        min_or_max, r2_high = self.min_or_max()
        for prediction_alg in self.prediction_algs:
            r2 = self.ml_family.get_SL_model_r2(cl_alg, parameters, cur_cluster, prediction_alg)

            print(f'Check R2 for {cl_alg}.{parameters}.{cur_cluster}.{prediction_alg} = {r2}')
            r2_high = min_or_max(r2_high, r2)
            if r2 is r2_high:
                if alg_high is not None:
                    self.ml_family.del_SL(cl_alg, parameters, cur_cluster, alg_high)
                alg_high = prediction_alg
            else:
                self.ml_family.del_SL(cl_alg, parameters, cur_cluster, prediction_alg)

    def perform_clustering_alg_with_clusters(self, cl_alg, parameters):
        """
        This performs a clustering algorithm using a given maximum number of clusters
        :param cl_alg: A clustering algorithm
        :param parameters:
        """

        #############################################################################
        # 1. Perform clustering algorithm using input training data
        print(f'perform_prediction_alg {cl_alg} with the cluster {parameters}')
        cl = Clustering_Alg()
        cl.set_algs(cl_alg)
        cl.set_base(parameters)
        cl.set_data(self.data_x, self.data_y, self.clustering_variables)
        cl.run()

        # Note that the clustering algorithm can change the number of clusters
        # e.g., ) The default n_clusters = 3 changes to n_clusters = 2 according to the clustering result
        self.ml_family.add_CL_model(cl_alg, parameters, cl)

        # Get clustered data from the cluster model
        data, data_x, data_y = cl.get_clustered_data_XY(cl_alg)

        #############################################################################
        # 2. Check cluster consistency:
        # If a cluster contains only one datum, data grouped by the cluster is determined as inconsistent data
        # Then, return it with a lowest score
        for cur_cluster, datum in data_x.items():
            if len(data_y[cur_cluster]) == 1:
                print(len(data_y[cur_cluster]))
            print(f'data split for {cl_alg}.{parameters}.{cur_cluster} X-Size[{len(data_x[cur_cluster])}] Y-Size[{len(data_y[cur_cluster])}]')

            # inconsistent clustered data!
            if len(data_x[cur_cluster]) != len(data_y[cur_cluster]):
                warnings.warn('inconsistent clustered data!')
                self.ml_family.set_CL_model_avg_r2(cl_alg, parameters, -10000)
                return

        #############################################################################
        # 3. Preparing SL learning
        thread = []

        for cur_cluster, datum in data_x.items():
            # split data for machine learning
            # x_train, x_val, y_train, y_val = train_test_split(data_x[cur_cluster], data_y[cur_cluster], test_size=test_size)

            if self.is_experiment:
                # print(f'sub-training data size[{len(y_train)}], validation data size[{len(y_val)}]')

                self.ml_family.set_CL_data(cl_alg, parameters, cur_cluster, 'x_train', data_x[cur_cluster])
                self.ml_family.set_CL_data(cl_alg, parameters, cur_cluster, 'y_train', data_y[cur_cluster])

            t = threading.Thread(target=self.perform_machine_learning, args=([cl_alg, parameters, cur_cluster, data_x[cur_cluster], data_y[cur_cluster]]))
            t.setDaemon(True)
            thread.append(t)

        for t in thread:
            t.start()

        for t in thread:
            t.join()

        #############################################################################
        # 4. Calculate the average R2 and store it to 'ml_models.cl_alg.parameters.avg_r2'
        avg_r2 = []
        cl_models = self.ml_family.get_CL_models(cl_alg, parameters)

        for cur_cluster, ml_alg_r2 in cl_models.items():
            try:
                r2 = list(ml_alg_r2.values())[0]['R2']
            except IndexError:
                print('list index out of range')
            avg_r2.append(r2)

        mean_avg_r2 = mean(avg_r2)
        self.ml_family.set_CL_model_avg_r2(cl_alg, parameters, mean_avg_r2)

    def perform_clustering(self, cl_alg):
        """
        This performs clustering
        :param cl_alg: A clustering algorithm
        """

        #############################################################################
        # 1. Perform clustering
        thread = []
        # for parameters in range(2, self.max_clusters + 1):
        for parameters in self.clustering_algs[cl_alg]:
            self.ml_family.add_CL_param(cl_alg, parameters)
            print(f'[Thread] clustering alg {cl_alg} with the cluster parameter {parameters} start')
            t = threading.Thread(target=self.perform_clustering_alg_with_clusters, args=([cl_alg, parameters]))
            t.setDaemon(True)
            thread.append(t)

        for t in thread:
            t.start()

        for t in thread:
            t.join()

        #############################################################################
        # 2. select a best number of clusters and remove all low-scored models
        cl_num_high = None
        min_or_max, avg_r2_high = self.min_or_max()

        # for parameters in range(2, self.max_clusters + 1):
        for parameters in self.clustering_algs[cl_alg]:
            avg_r2 = self.ml_family.get_CL_model_avg_r2(cl_alg, parameters)

            print(f'Check avg_r2 for {cl_alg}.{parameters} = {avg_r2}')
            avg_r2_high = min_or_max(avg_r2_high, avg_r2)
            if avg_r2 is avg_r2_high:
                if cl_num_high is not None:
                    self.ml_family.del_CL_param(cl_alg, cl_num_high)
                cl_num_high = parameters
            else:
                self.ml_family.del_CL_param(cl_alg, parameters)

        self.ml_family.set_CL_total_avg_r2(cl_alg, avg_r2_high)

    def run(self):
        """
        This is a main function for running DC-ML learning
        :return: An ML model family object
        """

        #############################################################################
        # 1. Perform clustering
        thread = []
        for cl_alg in self.clustering_algs:
            self.ml_family.add_CL(cl_alg)
            print(f'[Thread] {cl_alg} start')
            t = threading.Thread(target=self.perform_clustering, args=([cl_alg]))
            t.setDaemon(True)
            thread.append(t)

        for t in thread:
            t.start()

        for t in thread:
            t.join()

        #############################################################################
        # 2. perform ML for non-cluster models
        # 'NON-CLUSTER' is used for machine learning of SL models without clustering
        # self.ml_family.add_CL('NON-CLUSTER')
        # self.ml_family.add_CL_param('NON-CLUSTER', 1)
        # self.ml_family.add_CL_model('NON-CLUSTER', 1, 0)
        # self.ml_family.set_CL_data('NON-CLUSTER', 1, 0, 'x_train', self.data_x)
        # self.ml_family.set_CL_data('NON-CLUSTER', 1, 0, 'y_train', self.data_y)
        # self.perform_machine_learning('NON-CLUSTER', 1, 0, self.data_x, self.data_y)
        # cl_models = self.ml_family.get_CL_models('NON-CLUSTER', 1)
        # for cur_cluster, ml_alg_r2 in cl_models.items():
        #     r2 = list(ml_alg_r2.values())[0]['R2']
        # self.ml_family.set_CL_total_avg_r2('NON-CLUSTER', r2)

        #############################################################################
        # 3. select a high-scored model
        min_or_max, avg_r2_high = self.min_or_max()
        high_scored_cl = None
        cl_algs = list(self.ml_family.get_CL().keys())
        for cl_alg in cl_algs:
            avg_r2 = self.ml_family.get_CL_total_avg_r2(cl_alg)
            avg_r2_high = min_or_max(avg_r2_high, avg_r2)
            if avg_r2 is avg_r2_high:
                if high_scored_cl is not None:
                    self.ml_family.del_CL(high_scored_cl)

                high_scored_cl = cl_alg
            else:
                self.ml_family.del_CL(cl_alg)

        #############################################################################
        # 4. perform ML **again** using all data of both training and validation data sets
        sl_models = self.ml_family.get_sl_models()
        for cluster_id, sl in sl_models.items():
            # print(cluster_id, sl)
            x_train, y_train = self.ml_family.get_clustered_data_by_id(cluster_id)
            # self.data[cl_alg][parameters]

            # temporary name for CBN
            cbn_name = f'{cl_alg}_{cluster_id}_{datetime.now().strftime("%m_%d_%Y %H_%M_%S")}'

            if isinstance(sl['SL_MODEL'], str):
                sl_name = 'ContinuousBNRegressor'
            else:
                sl_name = type(sl['SL_MODEL']).__name__

            # perform ML
            model = self.do_machine_learning(sl_name, x_train, y_train, cbn_name=cbn_name)

            # Replace the old with the new SL model
            sl['SL_MODEL'] = model

        print('!!! An ML model family was selected !!!')

        return self.ml_family

    def perform_prediction(self, x_test, y_test, metrics='R2'):
        """
        This performs prediction given a test data set
        :param x_test: test data for X variables
        :param y_test: test data for a y variable
        :return: predicted y values, R2 scores
        """

        self.set_metrics(metrics)

        cl_alg = self.ml_family.get_clustering_alg()
        y_label = []

        # None-cl_model means that the Non-Cluster model was selected
        if cl_alg is not None:
            # Get data using clustering variables
            data_for_clustering = x_test[self.clustering_variables]

            cl_model = self.ml_family.get_cl_model()
            # normalize dataset for easier parameter selection
            x_test_norm = cl_model.data_scaler(data_for_clustering)
            # predict label using the normalized data
            y_label = cl_alg.predict(x_test_norm)

            index = 0
            yPredicted = []

            for cluster_id in y_label:
                ml_name, ml_model = self.ml_family.get_sl_model(cluster_id)
                yPre, r2 = self.do_prediction(ml_name, ml_model, x_test.iloc[[index]], y_test.iloc[[index]], cbn_name = datetime.now().strftime("%m_%d_%Y %H_%M_%S"))
                yPredicted.append(yPre)
                index += 1

            yPredicted2 = pd.DataFrame(yPredicted)
            r2 = self.score(y_test, yPredicted2)
        else:
            ml_name, ml_model = self.ml_family.get_sl_model(0)
            yPredicted, r2 = self.do_prediction(ml_name, ml_model, x_test, y_test, cbn_name=datetime.now().strftime("%m_%d_%Y %H_%M_%S"))

        return yPredicted, r2, y_label

In [11]:
import numpy as np
import pandas as pd


##########################################
# Step 1. Machine Learning
x_data = np.array([[667., 7], [693.3, 7], [732.9, 6], [658.9, 1], [702.8, 7], [667., 7], [693.3, 7], [732.9, 6], [658.9, 1], [702.8, 7], [667. , 7], [693.3, 7], [732.9, 6], [658.9, 1], [702.8, 7], [697.2, 1], [658.7, 2], [723.1, 1], [719.5, 3], [687.4, 1], [704.1, 1], [658.8, 4], [667.8, 3], [703.4, 3]])
y_data = np.array([[667.], [693.3], [732.9], [658.9], [702.8], [667.], [693.3], [732.9], [658.9], [702.8], [667.], [693.3], [732.9], [658.9], [702.8],[697.2], [658.7], [723.1], [719.5], [687.4], [704.1], [658.8], [667.8], [703.4]])

x_df = pd.DataFrame({'X1': x_data[:, 0], 'X2': x_data[:, 1]})
y_df = pd.DataFrame({'Y': y_data[:, 0]})

clustering_algs = {'GaussianMixture':[2, 3, 4], 'Birch':[2, 3, 4], 'MiniBatchKMeans':[2, 3, 4]}
prediction_algs = ['GaussianProcessRegressor', 'RandomForestRegressor']
selected_variables = ['X1']

dc_ml = DataClusterBasedMachineLearning(x_df, y_df, selected_variables, clustering_algs, prediction_algs)
ml_family = dc_ml.run()


##########################################
# Step 2. Prediction
x_test_data = np.array([[661., 7], [696.3, 8]])
y_test_data = np.array([[662.], [699.3]])
x_test_df = pd.DataFrame({'X1': x_test_data[:, 0], 'X2': x_test_data[:, 1]})
y_test_df = pd.DataFrame({'Y': y_test_data[:, 0]})

yPredicted, score, y_label = dc_ml.perform_prediction(x_test_df, y_test_df)

print(yPredicted)
print(f'Score = {score}')

[Thread] GaussianMixture start
[Thread] Birch start
[Thread] MiniBatchKMeans start
[Thread] clustering alg GaussianMixture with the cluster parameter 2 start
[Thread] clustering alg GaussianMixture with the cluster parameter 3 start
[Thread] clustering alg GaussianMixture with the cluster parameter 4 start
[Thread] clustering alg Birch with the cluster parameter 2 startperform_prediction_alg GaussianMixture with the cluster 2
[Thread] clustering alg MiniBatchKMeans with the cluster parameter 2 start
[Thread] clustering alg MiniBatchKMeans with the cluster parameter 3 start
[Thread] clustering alg MiniBatchKMeans with the cluster parameter 4 start
[Thread] clustering alg Birch with the cluster parameter 3 start
perform_prediction_alg MiniBatchKMeans with the cluster 2
mean[691.02916667], var[639.89289931], n_samples[24], scale[[25.29610443]][Thread] clustering alg Birch with the cluster parameter 4 start
perform_prediction_alg GaussianMixture with the cluster 3

perform_prediction_alg M

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_birch.py:638: ConvergenceWarning: Number of subclusters found (3) by Birch is less than (4). Decrease the threshold.
  % (len(centroids), self.n_clusters), ConvergenceWarning)


data split for Birch.2.1 X-Size[9] Y-Size[9]data split for Birch.4.0 X-Size[9] Y-Size[9]data split for GaussianMixture.2.1 X-Size[9] Y-Size[9]
data split for GaussianMixture.2.0 X-Size[15] Y-Size[15]
[Thread] GaussianMixture.2.1 -> ML alg GaussianProcessRegressor
[Thread] GaussianMixture.2.1 -> ML alg RandomForestRegressor


data split for Birch.2.0 X-Size[15] Y-Size[15]
[Thread] GaussianMixture.2.0 -> ML alg GaussianProcessRegressor
[Thread] GaussianMixture.2.0 -> ML alg RandomForestRegressor
[Thread] Birch.2.1 -> ML alg GaussianProcessRegressor
[Thread] Birch.2.1 -> ML alg RandomForestRegressor
data split for GaussianMixture.4.1 X-Size[4] Y-Size[4]
data split for GaussianMixture.4.2 X-Size[10] Y-Size[10]
data split for GaussianMixture.4.0 X-Size[5] Y-Size[5]
data split for GaussianMixture.4.3 X-Size[5] Y-Size[5]
[Thread] GaussianMixture.4.1 -> ML alg GaussianProcessRegressor
[Thread] GaussianMixture.4.1 -> ML alg RandomForestRegressor
data split for GaussianMixture.3.1 X-Size[9] Y-Si

Check R2 for MiniBatchKMeans.4.0.GaussianProcessRegressor = 3.447171039283603e-06Check R2 for MiniBatchKMeans.2.0.GaussianProcessRegressor = 3.447171039283603e-06
Check R2 for MiniBatchKMeans.4.0.RandomForestRegressor = 1.6820250000000982

Check R2 for MiniBatchKMeans.2.0.RandomForestRegressor = 1.786725000000152
Check R2 for MiniBatchKMeans.2.1.GaussianProcessRegressor = 6.129511019383114e-08
Check R2 for MiniBatchKMeans.2.1.RandomForestRegressor = 7.623705357143145
Check avg_r2 for MiniBatchKMeans.2 = 1.7542330747387171e-06
Check avg_r2 for MiniBatchKMeans.3 = 0.005336048271361923
Check avg_r2 for MiniBatchKMeans.4 = 0.017890596708891593
!!! An ML model family was selected !!!
[array([661.]), array([696.3])]
Score = 0.9856248517727629


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
